# ${\mathfrak{QMatch}}$

In this tutorial, we introduce the package $\mathfrak{QMatch}$ by 
- explaining functions 
- giving examples and some cross-checks

$\mathfrak{QMatch}$ is devoted to simulate $Gaussian$ $states$ and $Gaussian$ $operations$ for $1+1$ dimensional models with Hamiltonians that can be expressed in the form of free fermions. 

This package focuses on frequently-used quantities and operations for quantum states and computes/simulates them by applying the free fermion formulism. Thus, the central object of study is the correlation matrix, given which the fermionic Gaussian state can be completely determined. Functions in this package mainly work with the correlation matrices. 

See the following references for a thorough review on fermionic Gaussian states and Gaussian linear maps. 
- S. Bravyi, Lagrangian representation for fermionic linear optics, https://arxiv.org/abs/quant-ph/0404180
- B. G. Swingle and Y. Wang, Recovery map for fermionic Gaussian channels, J. Math. Phys. 60, 072202 (2019), https://arxiv.org/abs/1811.04956
- Appendix A of Y. Hu and Y. Zou, Petz map recovery for long-range entangled quantum many-body states 

## List of functions
- ground state correlation matrix: **GroundStateCorrMtx(M: np.ndarray) -> np.ndarray**
- ground state energy: **ground_state_energy(M: np.ndarray) -> float**
- tracing: **reduced_CorrMtx(Grho: np.ndarray, siteL: int, siteR: int) -> np.ndarray**
- tensor product: **tensor_prod(G1: np.ndarray, G2: np.ndarray) -> : np.ndarray**
- von Neumann Entropy: **vn_entropy(Grho: np.ndarray) -> float**
- Conditional mutual information: **CMI(Grho: np.ndarray, La: int, Lb: int, Lc: int) -> float**
- Fidelity: **Fidelity(Grho: np.ndarray, Gsigma: np.ndarray) -> float**
- Gaussian channel: **Gaussian_channel(A: np.ndarray, B: np.ndarray, Grho: np.ndarray) -> : np.ndarray**
- Erasure channel: **erasure_channel(Grho: np.ndarray, L1: int, L2: int) -> np.ndarray**
- Petz recovery map:
- roated Petz recovery map:
- Z measurement: 

# Import QMatch

In [1]:
from QMatch import *

## import other packages needed for demonstrations

In [2]:
import numpy as np
from numpy import linalg as LA

# Fermionic Gaussian States

## Correlation matrix

Given a free fermion Hamiltonian taking the following form,
\begin{equation}
H ~=~ \frac{i}{2}\,\sum_{i,j=1}^{2L}\,M_{ij}\,\gamma_i\,\gamma_j
\end{equation}
Function 
**GroundStateCorrMtx(M: np.ndarray) -> np.ndarray**
computes the correlation matrix for the ground state of $H$

Input: 
- the matrix $M$: a $2L\times 2L$ dimensional real and antisymmetric matrix. 

Output: 
- the correlation matrix for the ground state of $H$: np.ndarray

### Example
Here we compute the correlation matrix of critical Ising ground state with the system size $L$, thus the output is a $2L\times 2L$ antisymmetric matrix. 
The Hamiltonian reads
$$
H_{\rm Ising} ~=~ \frac{i}{2} \sum_{k=1}^{2L} \Big[ \gamma_k \gamma_{k+1} - \gamma_{k+1} \gamma_k \Big] 
$$
with periodic boundary condition $\gamma_{2L+1} = - \gamma_1$. 

The matrix $M$ can be read off from the above equation and the function $\color{purple}{\rm Ising\_Hamiltonian\_M}(L)$ has been built to compute $M$. 

In [12]:
Ising_Hamiltonian_M(L = 3)

array([[ 0.,  1.,  0.,  0.,  0.,  1.],
       [-1.,  0.,  1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  1.],
       [-1.,  0.,  0.,  0., -1.,  0.]])

Thus, the correlation matrix in this case can be obtained as follows.

In [13]:
GroundStateCorrMtx(Ising_Hamiltonian_M(L = 3))

array([[ 0.        , -0.66666667,  0.        , -0.33333333,  0.        ,
        -0.66666667],
       [ 0.66666667,  0.        , -0.66666667,  0.        , -0.33333333,
         0.        ],
       [ 0.        ,  0.66666667,  0.        , -0.66666667,  0.        ,
        -0.33333333],
       [ 0.33333333,  0.        ,  0.66666667,  0.        , -0.66666667,
         0.        ],
       [ 0.        ,  0.33333333,  0.        ,  0.66666667,  0.        ,
        -0.66666667],
       [ 0.66666667,  0.        ,  0.33333333,  0.        ,  0.66666667,
         0.        ]])

### Cross-check
To check correlation matrix of critical Ising ground state, we compare the result obtained above with the analytic result, which is 
$$
G^{\rm Ising}_{jl} ~=~ 
    \begin{cases}
        0 \qquad\qquad\qquad\qquad~ j-l~\text{is even} \\
        1/\left[L\, \sin\frac{\pi}{2L}(j-l) \right] ~~ j-l~\text{is odd} \\
    \end{cases}
$$
Again, the function $\color{purple}{\rm IsingGS\_CorrMtx}(L)$ has been built to compute $G^{\rm Ising}$. 

In [7]:
np.allclose(GroundStateCorrMtx(Ising_Hamiltonian_M(L=3)), IsingGS_CorrMtx(L=3))

True

## Ground state energy
Function **ground_state_energy(M: np.ndarray) -> float** computes the ground state energy. 

Input: 
- the matrix $M$ in the Hamiltonian $$H ~=~ \frac{i}{2}\,\sum_{i,j=1}^{2L}\,M_{ij}\,\gamma_i\,\gamma_j$$
- it is a $2L\times 2L$ dimensional real and antisymmetric matrix

Output: 
- the ground state energy $E_{\rm GS}$

### Example
the ground state energy for critical Ising with system size $L$ can be computed as follows

In [17]:
ground_state_energy(Ising_Hamiltonian_M(L = 3))

np.float64(-4.0)

### Cross-check
To check the ground state energy result obtained above, we compare it with the result computed by diagonalization. 

Recall that the Hamiltonian for the Ising model reads
$$
H_{\rm Ising} ~=~ -\,\sum_{i=1}^L\left( X_i\,X_{i+1} + g\,Z_i \right)
$$
where $X$ and $Z$ are Pauli matrices and the criticality occurs at $g=1$. This $2^L\times 2^L$ matrix $H_{\rm Ising}$ can be computed by the built-in function $\color{purple}{\rm Ising\_H\_def}(L,g)$. 

The ground state energy is the smallest eigenvalue of $H$. Namely, 

In [19]:
eigv = LA.eigvalsh(Ising_H_def(L=3, g=1))
eigv[0]

np.float64(-3.9999999999999996)

## Tracing 
Function
**reduced_CorrMtx(Grho: np.ndarray, siteL: int, siteR: int) -> np.ndarray**
computes the correlation matrix $G^{\rho_A}$ for a reduced density matrix $\rho_A$
$$
\rho_A ~=~ {\rm Tr}_A\,\rho
$$

Inputs: 
- Grho: the correlation matrix $G^{\rho}$ for $\rho$, which is a $2L\times2L$ real, antisymmetric matrix, namely the system is composed by $L$ spins
- siteL: the number of spins sitting on the left of $A$, siteL $\in [0,L]$
- siteR: the region $A$ ends at the siteR-th spin, siteR $\in [0,L]$ (siteR $\ge$ siteL)

Output: 
- the correlation matrix for $\rho_A$

### Example
Consider the critical Ising ground state with $L = 3$ and subregion $A$ is the first spin. 
Thus the reduced correlation matrix should be the first $2x2$ block of $G^{\rho}$, i.e. 
$$ G^{\rho_A} ~=~ 
\begin{pmatrix}
0 & -2/3\\
2/3 & 0 
\end{pmatrix} ~.
$$
We can see $G^{\rho_A}$ is reproduced as follows

In [22]:
G_rho = GroundStateCorrMtx(Ising_Hamiltonian_M(L = 3))
reduced_CorrMtx(G_rho, 0, 1)

array([[ 0.        , -0.66666667],
       [ 0.66666667,  0.        ]])

## Tensor product
Function
**tensor_prod(G1: np.ndarray, G2: np.ndarray) -> : np.ndarray**
computes the correlation matrix for a tensor product state. 

Inputs:
- G1: the correlation matrix $G^{\rho_1}$ for $\rho_1$
- G2: the correlation matrix $G^{\rho_2}$ for $\rho_2$

Output: 
- the correlation matrix $G^{\rho_1\otimes\rho_2}$ for $\rho_1\otimes\rho_2$

### Example
consider 
- $\rho_1$ is the reduced density matrix $\rho_A$ for critical Ising ground state with $L=3$ and subregion $A$ is the first spin. Namely, 
$$ G^{\rho_1} ~=~ 
\begin{pmatrix}
0 & -2/3\\
2/3 & 0 
\end{pmatrix}
$$
- $\rho_2 = \frac{1}{2}\mathbb{I}_2$. Thus $G^{\rho_2}$ is a zero matrix, 
$$
G^{\rho_2} ~=~ 
\begin{pmatrix}
0 & 0\\
0 & 0 
\end{pmatrix}
$$

The result should be 
$$
G^{\rho_1\otimes \rho_2} ~=~ 
\begin{pmatrix}
0 & -2/3 & 0 & 0\\
2/3 & 0  & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}
$$
which is reproduced as follows

In [23]:
G_rho = GroundStateCorrMtx(Ising_Hamiltonian_M(L = 3))
G1 = reduced_CorrMtx(G_rho, 0, 1)
G2 = np.zeros([2,2])
tensor_prod(G1, G2)

array([[ 0.        , -0.66666667,  0.        ,  0.        ],
       [ 0.66666667,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

## von Neumann Entropy
Function 
**vn_entropy(Grho: np.ndarray) -> float** 
computes the von Neumann Entropy. 

Input: 
- Grho: the correlation matrix $G^{\rho}$ for $\rho$, which is a real, antisymmetric matrix

Output: 
- the von Neumann Entropy for the state $\rho$, i.e.
$$S(\rho)=-{\rm Tr}(\rho\log_2\rho)$$
Note that the logarithm is base 2. 

### Example
Consider $\rho$ to be the crtical Ising ground state with $L=10$ spins, and its reduced density matrix $\rho_A$ where $L_A = 5$. Now we compute their von Neumann entropy. First, their correlation matrices are 

In [3]:
G_rho = GroundStateCorrMtx(Ising_Hamiltonian_M(L = 10))
G_rho_A = reduced_CorrMtx(G_rho, 0, 5)

Since now $\rho$ is a pure state, it has vanishing entropy.

In [4]:
vn_entropy(G_rho)

0

For $\rho_A$, it has nonzero entropy. 

In [5]:
vn_entropy(G_rho_A)

np.float64(0.9695123579328397)

### Cross-check 
To check $S(\rho_A)$, we compare with the result obtained by the exact diagonalization as follows, where the function $\color{purple}{\rm Ising\_SvN\_exact\_diag}(L\_A, L)$ computes $S(\rho_A)$ by using the ground state we have obtained by diagonalizing the Hamiltonian and then plugging in the definition of the entropy. 

In [8]:
Ising_SvN_exact_diag(L_A = 5, L = 10)

np.float64(0.9695123528042153)

## Conditional mutual information
Given the correlation function $G^{\rho_{ABC}}$ and the subsystem configuration, the function
**CMI(Grho: np.ndarray, La: int, Lb: int, Lc: int) -> float**
computes the conditional mutual information (CMI) $I(A:C|B)$
$$
I(A:C|B) ~=~ S(\rho_{AB}) ~+~ S(\rho_{BC}) ~-~ S(\rho_B) ~-~ S(\rho_{ABC})
$$
where contiguous subregions $A$, $B$, $C$ have lengths $L_A$, $L_B$, and $L_C$ respectively. $A$ starts from the first spin. 

Inputs:
- Grho: the correlation function $G^{\rho_{ABC}}$ 
- La: size of the subregion $A$, $L_A$
- Lb: size of the subregion $B$, $L_B$
- Lc: size of the subregion $C$, $L_C$

Output:
- the CMI $I(A:C|B)$

### Example
Consider $\rho$ to be the crtical Ising ground state with $L=10$ spins and $(L_A,L_B,L_C) = (2,6,2)$. Then its CMI reads

In [3]:
G_rho = GroundStateCorrMtx(Ising_Hamiltonian_M(L = 10))
CMI(G_rho, 2, 6, 2)

np.float64(0.7246255063844171)

If $(L_A,L_B,L_C) = (1,6,1)$, we need to first compute $G^{\rho_{ABC}}$. Namely,

In [5]:
G_rhoABC = reduced_CorrMtx(G_rho, 0, 1+6+1)
CMI(G_rhoABC, 1, 6, 1)

np.float64(0.03833429119740073)

## Fidelity
Function
**Fidelity(Grho: np.ndarray, Gsigma: np.ndarray) -> float**
computes the fidelity $F(\rho,\sigma)$ between two quantum states $\rho$ and $\sigma$. 

Inputs:
- Grho: the correlation matrix of $\rho$
- Gsigma: the correlation matrix of $\sigma$

Output:
- the fidelity $F(\rho,\sigma)$

### Example
Consider $\rho$ to be the crtical Ising ground state with $L=8$ spins, the fidelity $F(\rho_{AB}, \rho_{BC})$ with $(L_A,L_B,L_C) = (2,2,4)$ reads

In [7]:
G_rho = GroundStateCorrMtx(Ising_Hamiltonian_M(L = 8))
Fidelity(reduced_CorrMtx(G_rho, 0, 4), reduced_CorrMtx(G_rho, 4, 8))

np.float64(1.0000000000160187)

# Fermionic Gaussian Channel
A fermionic Gaussian channel $\mathcal{E}$ admits the following Grassmann representation 
$$
\omega\left( {\cal E}(X), \theta\right) ~=~ \int D\mu D\eta\, \exp\Big[ \frac{i}{2}\,\theta^{\rm T}A\,\theta ~+~ i\,\theta^{\rm T}B\,\eta \,+\, i\eta^{\rm T}\mu \Big]\,\omega(X,\mu)
$$
where 
- $X$ is the input fermionic state composed of $2L$ Majorana fermions or equivalently $L$ spins. 
- $A$ and $B$ are antisymmetric $2L\times 2L$ matrices. The fermionic Gaussian channel can be completely determined by $A$ and $B$. 

Function
**Gaussian_channel(A: np.ndarray, B: np.ndarray, Grho: np.ndarray) -> : np.ndarray**
computes the new correlation matrix $G^{\mathcal{E}(\rho)}$ after a Gaussian linear map $\mathcal{E}$ acting on $\rho$. 

Inputs: 
- Grho: the original correlation matrix $G^{\rho}$ for $\rho$
- A, B: matrices in the integral representation of the fermionic Gaussian channel $\mathcal{E}$

Output: 
- the new correlation matrix $G^{\mathcal{E}(\rho)}$

## Erasure channel
Consider 

Function 
**erasure_channel(Grho: np.ndarray, L1: int, L2: int) -> np.ndarray**
yields the 

### Example


# Gaussian Measurements